# Water Quality Prediction: Benchmark Notebook 

## Challenge Overview

Welcome to the EY AI & Data Challenge 2026!  
The objective of this challenge is to build a robust **machine learning model** capable of predicting water quality across various river locations in South Africa. In addition to accurate predictions, the model should also identify and emphasize the key factors that significantly influence water quality.

Participants will be provided with a dataset containing three water quality parameters — **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus** — collected between 2011 and 2015 from approximately 200 river locations across South Africa. Each data point includes the geographic coordinates (latitude and longitude) of the sampling site, the date of collection, and the corresponding water quality measurements.

Using this dataset, participants are expected to build a machine learning model to predict water quality parameters for a separate validation dataset, which includes locations from different regions not present in the training data. The challenge also encourages participants to explore feature importance and provide insights into the factors most strongly associated with variations in water quality.

This challenge is designed for participants with varying levels of experience in data science, remote sensing, and environmental analytics. It offers a valuable opportunity to apply machine learning techniques to real-world environmental data and contribute to advancing water quality monitoring using artificial intelligence.


**About the Notebook:**  

In this notebook, we demonstrate a basic workflow that serves as a foundation for the challenge. The model has been developed to predict **water quality parameters** using features derived from the **Landsat** and **TerraClimate** datasets. Specifically, four spectral bands — **SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1) — were utilized from Landsat, along with derived spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). In addition, the **PET** (Potential Evapotranspiration) variable was incorporated from the **TerraClimate** dataset to account for climatic influences on water quality.

The dataset spans a five-year period from **2011 to 2015**. Using **API-based data extraction** methods, both Landsat and TerraClimate features were retrieved directly from the [Microsoft Planetary Computer portal](https://planetarycomputer.microsoft.com).

These combined spectral, index-based, and climatic features were used as predictors in a regression model to estimate three key water quality parameters: **Total Alkalinity (TA)**, **Electrical Conductance (EC)**, and **Dissolved Reactive Phosphorus (DRP)**.

Please note that this notebook serves only as a starting point. Several assumptions were made during the data extraction and model development process, which you may find opportunities to improve upon. Participants are encouraged to explore additional features, enhance preprocessing techniques, or experiment with different regression algorithms to optimize predictive performance.


## Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process.

In [ ]:
# %pip install uv
# !uv pip install -r requirements.txt

In [1]:
# ============================================
# Setup: Import Libraries and Create Session
# ============================================

# Snowflake session
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Planetary Computer tools for STAC API access and authentication
# NOTE: These packages are not available in Snowflake Anaconda
# import pystac_client
# import planetary_computer as pc
# from odc.stac import stac_load
# from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os

print("✅ All libraries imported successfully!")
print("✅ Snowflake session created!")

## Response Variable

Before building the model, we first load the **water quality training dataset**. The curated dataset contains samples collected from various monitoring stations across the study region. Each record includes the geographical coordinates (Latitude and Longitude), the sample collection date, and the corresponding **measured values** for the three key water quality parameters — **Total Alkalinity (TA)**, **Electrical Conductance (EC)**, and **Dissolved Reactive Phosphorus (DRP)**.


In [ ]:
# ============================================
# Set Database and Schema Context
# ============================================
print("=" * 80)
print("🔧 SETTING SNOWFLAKE CONTEXT")
print("=" * 80)

# Set the database and schema
session.use_database("EY_WATER_QUALITY")
session.use_schema("CHALLENGE")

print("\n✅ Context set:")
print(f"   Database: EY_WATER_QUALITY")
print(f"   Schema: CHALLENGE")
print("=" * 80)

In [ ]:
# ============================================
# Load all training datasets from Snowflake tables
# ============================================
print("\n📊 Loading data from Snowflake tables...")

# Ensure we're in the correct database and schema
session.use_database("EY_WATER_QUALITY")
session.use_schema("CHALLENGE")

# Water quality training data (target variables)
Water_Quality_df = session.table("WATER_QUALITY_TRAINING").to_pandas()
print(f"✅ Water Quality Training: {Water_Quality_df.shape[0]:,} rows × {Water_Quality_df.shape[1]} columns")

# Landsat features (satellite data)
Landsat_Features_df = session.table("LANDSAT_FEATURES_TRAINING").to_pandas()
print(f"✅ Landsat Features Training: {Landsat_Features_df.shape[0]:,} rows × {Landsat_Features_df.shape[1]} columns")

# TerraClimate features (climate data)
TerraClimate_Features_df = session.table("TERRACLIMATE_FEATURES_TRAINING").to_pandas()
print(f"✅ TerraClimate Features Training: {TerraClimate_Features_df.shape[0]:,} rows × {TerraClimate_Features_df.shape[1]} columns")

# Validation datasets
Landsat_Features_val_df = session.table("LANDSAT_FEATURES_VALIDATION").to_pandas()
print(f"✅ Landsat Features Validation: {Landsat_Features_val_df.shape[0]:,} rows × {Landsat_Features_val_df.shape[1]} columns")

TerraClimate_Features_val_df = session.table("TERRACLIMATE_FEATURES_VALIDATION").to_pandas()
print(f"✅ TerraClimate Features Validation: {TerraClimate_Features_val_df.shape[0]:,} rows × {TerraClimate_Features_val_df.shape[1]} columns")

print("\n✨ All datasets loaded successfully!")

## Predictor Variables

Now that we have our water quality dataset, the next step is to gather the predictor variables from the **Landsat** and **TerraClimate** datasets. In this notebook, we demonstrate how to **load previously extracted satellite and climate data** from separate files, rather than performing the extraction directly, which allows for a smoother and faster experience. Participants can refer to the dedicated extraction notebooks—one for Landsat and another for TerraClimate—to understand how the data was retrieved and processed, and they can also generate their own output CSV files if needed. Using these pre-extracted CSV files, this notebook focuses on loading the predictor features and running the subsequent analysis and model training efficiently.

For more detailed guidance on the original data extraction process, you can review the Landsat and TerraClimate example notebooks available on the Planetary Computer portal:

- [Landsat-c2-l2 - Example-Notebook](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2#Example-Notebook)  
- [Terraclimate - Example-Notebook](https://planetarycomputer.microsoft.com/dataset/terraclimate#Example-Notebook)

We have used selected spectral bands — **SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1) — and computed key spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). These features capture surface moisture, vegetation, and water content characteristics that influence water quality variability.

In addition to Landsat features, we also incorporated the **Potential Evapotranspiration (PET)** variable from the **TerraClimate** dataset, which provides high-resolution global climate data. The PET feature captures the atmospheric demand for moisture, representing climatic conditions such as temperature, humidity, and radiation that influence surface water evaporation and thus affect water quality parameters.

The predictor features include:

- **SWIR22** – Sensitive to surface moisture and turbidity variations in water bodies.  
- **NIR** – Helps in identifying vegetation and suspended matter in water.  
- **Green** – Useful for detecting water color and surface reflectance changes.  
- **SWIR16** – Provides information on surface dryness and sediment concentration.  
- **NDMI** – Derived from NIR and SWIR16, indicates moisture and vegetation–water interaction.  
- **MNDWI** – Derived from Green and SWIR22, effective for distinguishing open water areas and reducing built-up noise.  
- **PET** – Extracted from the TerraClimate dataset, represents potential evapotranspiration influencing hydrological and water quality dynamics.


### **Tip 1**

Participants are encouraged to experiment with different combinations of **Landsat** bands or even include data from other public satellite data sources. By creating mathematical combinations of bands, you can derive various spectral indices that capture surface and environmental characteristics.


### Loading Pre-Extracted Landsat Data

In this notebook, we **load previously extracted Landsat data** from CSV files generated in a separate extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.

Participants are expected to generate their own data extraction CSV files by running the dedicated Landsat extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how individual bands and indices like **NDMI** were computed. Using these pre-extracted CSV files simplifies preprocessing and is ideal for large-scale environmental and water quality analysis.


### **Tip 2**

In the data extraction process (performed in the dedicated extraction notebooks), a 100 m focal buffer was applied around each sampling location rather than using a single point. Participants may explore creating different focal buffers around the locations (e.g., 50 m, 150 m, etc.) during extraction. For example, if a 50 m buffer was used for “Band 2”, the extracted CSV values would reflect the average of Band 2 within 50 meters of each location. This approach can help reduce errors associated with spatial autocorrelation.


In [ ]:
# ============================================
# Loading Pre-Extracted Landsat Data
# ============================================


# Load Landsat features from Snowflake table (already pre-extracted)
landsat_train_features = session.table("LANDSAT_FEATURES_TRAINING").to_pandas()

print(f"\n✅ Landsat training features loaded: {landsat_train_features.shape[0]:,} rows × {landsat_train_features.shape[1]} columns")

print("\n📋 Landsat feature columns:")
print(landsat_train_features.columns.tolist())

print("\nSample data (first 5 rows):")
display(landsat_train_features.head(5))

print("\n" + "=" * 80)
print("The predictor features used in this benchmark are:")
print("=" * 80)

print("\n  • SWIR22 - Sensitive to surface moisture and turbidity")
print("  • NIR - Identifies vegetation and suspended matter")
print("  • Green - Detects water color and surface reflectance")
print("  • SWIR16 - Surface dryness and sediment concentration")
print("  • NDMI - Normalized Difference Moisture Index (NIR & SWIR16)")
print("  • MNDWI - Modified Normalized Difference Water Index (Green & SWIR22)")
print("  • PET - Potential Evapotranspiration from TerraClimate")

print("\n" + "=" * 80)

In [ ]:
# ============================================
# Data Type Conversion for NDMI and MNDWI
# ============================================
print("=" * 80)
print("🔧 DATA TYPE CONVERSION")
print("=" * 80)

print("\nConverting NDMI and MNDWI columns to float data type to ensure proper")
print("numerical operations during model training.")

# Check if columns exist before conversion
if 'NDMI' in landsat_train_features.columns:
    print("\n✓ Converting NDMI to float...")
    landsat_train_features['NDMI'] = landsat_train_features['NDMI'].astype(float)
    print(f"  NDMI dtype: {landsat_train_features['NDMI'].dtype}")
else:
    print("\n⚠️  NDMI column not found in dataset")

if 'MNDWI' in landsat_train_features.columns:
    print("\n✓ Converting MNDWI to float...")
    landsat_train_features['MNDWI'] = landsat_train_features['MNDWI'].astype(float)
    print(f"  MNDWI dtype: {landsat_train_features['MNDWI'].dtype}")
else:
    print("\n⚠️  MNDWI column not found in dataset")

print("\n" + "=" * 80)
print("✅ Data type conversion complete!")
print("=" * 80)

# Display data types of all columns
print("\n📋 All column data types:")
print(landsat_train_features.dtypes)

print("\n📊 Updated dataset info:")
print(f"Shape: {landsat_train_features.shape[0]:,} rows × {landsat_train_features.shape[1]} columns")

# Show sample after conversion
print("\nSample data after conversion (first 5 rows):")
display(landsat_train_features.head(5))

### Loading Pre-Extracted TerraClimate Data

In this notebook, we **load previously extracted TerraClimate data** from CSV files generated in a dedicated extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.

Participants are expected to generate their own data extraction CSV files by running the dedicated TerraClimate extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how climate variables such as **Potential Evapotranspiration (PET)** were extracted. Using these pre-extracted CSV files ensures consistent, automated retrieval of high-resolution climate data that can be easily integrated with satellite-derived features for comprehensive environmental and hydrological analysis.


In [ ]:
# ============================================
# Loading Pre-Extracted TerraClimate Data
# ============================================


print("\n" + "=" * 80)

# Load TerraClimate features from Snowflake table (already pre-extracted)
Terraclimate_df = session.table("TERRACLIMATE_FEATURES_TRAINING").to_pandas()

print(f"\n TerraClimate training features loaded: {Terraclimate_df.shape[0]:,} rows × {Terraclimate_df.shape[1]} columns")

print("\n TerraClimate feature columns:")
print(Terraclimate_df.columns.tolist())

print("\nSample data (first 5 rows):")
display(Terraclimate_df.head(5))

print("\n" + "=" * 80)
print("Key TerraClimate Feature:")
print("=" * 80)

print("\n  • PET (Potential Evapotranspiration) - Represents the atmospheric demand for")
print("    moisture, capturing climatic conditions such as temperature, humidity, and")
print("    radiation that influence surface water evaporation and affect water quality")
print("    parameters.")

print("\n" + "=" * 80)
print(" TerraClimate data loading complete!")
print("=" * 80)

# Display summary statistics
print("\n TerraClimate Data Summary:")
display(Terraclimate_df.describe())

## Joining the Predictor Variables and Response Variables

Now that we have extracted our predictor variables, we need to join them with the response variables. We use the **combine_two_datasets** function to merge the predictor variables and response variables. The **concat** function from pandas is particularly useful for this step.


In [5]:
# ============================================
# Helper Function: Combine Two Datasets
# ============================================


# Combine two datasets vertically (along columns) using pandas concat function
def combine_two_datasets(dataset1, dataset2, dataset3):
    """
    Returns a vertically concatenated dataset.
    Attributes:
        dataset1 - Dataset 1 to be combined
        dataset2 - Dataset 2 to be combined
        dataset3 - Dataset 3 to be combined
    """
    data = pd.concat([dataset1, dataset2, dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

print("✅ Helper function 'combine_two_datasets' defined successfully!")
print("\nThis function will:")
print("  • Concatenate datasets horizontally (column-wise)")
print("  • Remove any duplicate columns")
print("  • Return a single unified dataset")

print("\n" + "=" * 80)

In [6]:
print("=" * 80)
print("🔀 JOINING PREDICTOR AND RESPONSE VARIABLES")
print("=" * 80)

print("\nCombining ground data (water quality) and predictor data (Landsat + TerraClimate)")
print("into a single dataset for model training.")

print("\n" + "=" * 80)

# Combining ground data and final data into a single dataset
wq_data = combine_two_datasets(Water_Quality_df, landsat_train_features, Terraclimate_df)

print(f"\n✅ Combined dataset created: {wq_data.shape[0]:,} rows × {wq_data.shape[1]} columns")

print("\n📋 All columns in combined dataset:")
for i, col in enumerate(wq_data.columns, 1):
    print(f"  {i}. {col}")

print("\n" + "=" * 80)
print("Sample of combined dataset (first 5 rows):")
print("=" * 80)
display(wq_data.head(5))

print("\n" + "=" * 80)
print("✅ Dataset joining complete!")
print("=" * 80)

print("\nThe combined dataset now contains:")
print("  • Water Quality measurements (TA, EC, DRP) - Response variables")
print("  • Landsat features (SWIR22, NIR, Green, SWIR16, NDMI, MNDWI) - Predictor variables")
print("  • TerraClimate features (PET) - Predictor variable")
print("  • Geographical coordinates (Latitude, Longitude)")
print("  • Sample dates")

print(f"\nThis unified dataset is ready for:")
print("  • Feature engineering")
print("  • Train/test split")
print("  • Model training")

print("\n" + "=" * 80)

### Handling Missing Values

Before model training, missing values in the dataset were carefully handled to ensure data consistency and prevent model bias. Numerical columns were imputed using their median values, maintaining the overall data distribution while minimizing the impact of outliers.


In [7]:
# ============================================
# Handling Missing Values
# ============================================
print("=" * 80)
print("🔧 HANDLING MISSING VALUES")
print("=" * 80)

print("\nBefore model training, missing values in the dataset were carefully handled to")
print("ensure data consistency and prevent model bias. Numerical columns were imputed")
print("using their median values, maintaining the overall data distribution while")
print("minimizing the impact of outliers.")

print("\n" + "=" * 80)

# Check for missing values before imputation
print("\n📊 Missing values BEFORE imputation:")
missing_before = wq_data.isna().sum()
missing_before_df = pd.DataFrame({
    'Column': missing_before.index,
    'Missing Count': missing_before.values,
    'Missing %': (missing_before.values / len(wq_data) * 100).round(2)
})
missing_before_df = missing_before_df[missing_before_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_before_df) > 0:
    print(f"\n⚠️  Found {len(missing_before_df)} columns with missing values:")
    display(missing_before_df)
else:
    print("\n✅ No missing values found!")

print("\n" + "=" * 80)

# Check data types before imputation
print("\n🔍 Checking data types...")
print("\nData types of key columns:")
for col in ['NDMI', 'MNDWI', 'NIR', 'GREEN', 'SWIR16', 'SWIR22', 'PET']:
    if col in wq_data.columns:
        print(f"  {col}: {wq_data[col].dtype}")

# Convert problematic columns to numeric (coerce errors to NaN)
print("\n🔄 Converting columns to numeric type...")
numeric_columns = ['NDMI', 'MNDWI', 'NIR', 'GREEN', 'SWIR16', 'SWIR22', 'PET',
                  'TOTAL_ALKALINITY', 'ELECTRICAL_CONDUCTANCE', 'DISSOLVED_REACTIVE_PHOSPHORUS']

for col in numeric_columns:
    if col in wq_data.columns:
        wq_data[col] = pd.to_numeric(wq_data[col], errors='coerce')
        print(f"  ✓ Converted {col} to numeric")

print("\n" + "=" * 80)

# Now impute missing values with median for numerical columns
print("\n🔄 Imputing missing values using median...")
wq_data = wq_data.fillna(wq_data.median(numeric_only=True))

print("✅ Imputation complete!")

# Verify missing values after imputation
print("\n" + "=" * 80)
print("\n📊 Missing values AFTER imputation:")
missing_after = wq_data.isna().sum()

if missing_after.sum() == 0:
    print("\n✅ No missing values remaining - Dataset is clean!")
else:
    remaining = missing_after[missing_after > 0]
    print(f"\n⚠️  Warning: {missing_after.sum()} missing values still remain:")
    display(pd.DataFrame({
        'Column': remaining.index,
        'Missing Count': remaining.values
    }))

print("\n" + "=" * 80)
print("Final Data Types:")
print("=" * 80)
for col in numeric_columns:
    if col in wq_data.columns:
        print(f"  {col}: {wq_data[col].dtype}")

print("\n" + "=" * 80)
print("Dataset Summary:")
print("=" * 80)
print(f"Shape: {wq_data.shape[0]:,} rows × {wq_data.shape[1]} columns")
print(f"Total missing values: {wq_data.isna().sum().sum()}")

print("\n✅ Data is now ready for train/test split and model training!")
print("=" * 80)

## Model Building

Now let us select the columns required for our model-building exercise. We will consider only **SWIR22**, **NDMI**, and **MNDWI** from the Landsat data, and **PET** from the TerraClimate dataset as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the water quality parameters.


In [ ]:
# ============================================
# MODEL BUILDING - FEATURE SELECTION
# ============================================

# ============================================
# Step 1: Check if wq_data exists, if not recreate it
# ============================================
print("\n🔍 Checking if wq_data exists...")

try:
    print(f"✓ wq_data found: {wq_data.shape}")
except NameError:
    print("⚠️  wq_data not found. Recreating from Snowflake tables...")
    
    # Load all three datasets from Snowflake
    print("\n📊 Loading datasets from Snowflake...")
    
    Water_Quality_df = session.table("WATER_QUALITY_TRAINING").to_pandas()
    print(f"  ✓ Water Quality: {Water_Quality_df.shape}")
    
    landsat_train_features = session.table("LANDSAT_FEATURES_TRAINING").to_pandas()
    print(f"  ✓ Landsat Features: {landsat_train_features.shape}")
    
    Terraclimate_df = session.table("TERRACLIMATE_FEATURES_TRAINING").to_pandas()
    print(f"  ✓ TerraClimate Features: {Terraclimate_df.shape}")
    
    # Combine datasets
    print("\n🔗 Combining datasets...")
    wq_data = pd.concat([Water_Quality_df, landsat_train_features, Terraclimate_df], axis=1)
    wq_data = wq_data.loc[:, ~wq_data.columns.duplicated()]
    
    # Fill missing values
    print("🔧 Handling missing values...")
    numeric_columns = wq_data.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        wq_data[col] = pd.to_numeric(wq_data[col], errors='coerce')
    wq_data = wq_data.fillna(wq_data.median(numeric_only=True))
    
    print(f"✅ wq_data created: {wq_data.shape}")

print("\n" + "=" * 80)

# ============================================
# Step 2: Show available columns
# ============================================
print("\n📋 Available columns in wq_data:")
print(wq_data.columns.tolist())

print("\n" + "=" * 80)

# ============================================
# Step 3: Select required columns
# ============================================
print("\n🎯 Selecting predictor and target variables...")

required_cols = ['SWIR22', 'NDMI', 'MNDWI', 'PET', 
                'TOTAL_ALKALINITY', 'ELECTRICAL_CONDUCTANCE', 
                'DISSOLVED_REACTIVE_PHOSPHORUS']

# Check which columns are available
available_cols = [col for col in required_cols if col in wq_data.columns]
missing_cols = [col for col in required_cols if col not in wq_data.columns]

if missing_cols:
    print(f"\n⚠️  Warning: Missing columns: {missing_cols}")
    print("\n🔍 Searching for similar column names...")
    for missing in missing_cols:
        similar = [c for c in wq_data.columns 
                  if missing.lower() in c.lower() or c.lower() in missing.lower()]
        if similar:
            print(f"  {missing} → Found similar: {similar}")
            # Use the first similar column found
            if similar[0] not in available_cols:
                available_cols.append(similar[0])
                print(f"    ✓ Using '{similar[0]}' instead")

# Select only available columns
wq_data = wq_data[available_cols]

print(f"\n✅ Feature selection complete!")
print(f"\nFinal dataset shape: {wq_data.shape[0]:,} rows × {wq_data.shape[1]} columns")

print("\n" + "=" * 80)

# ============================================
# Step 4: Categorize columns
# ============================================
print("\n📋 Selected columns:")

predictor_cols = [col for col in wq_data.columns 
                 if col not in ['TOTAL_ALKALINITY', 'ELECTRICAL_CONDUCTANCE', 'DISSOLVED_REACTIVE_PHOSPHORUS']]

target_cols = [col for col in wq_data.columns 
              if col in ['TOTAL_ALKALINITY', 'ELECTRICAL_CONDUCTANCE', 'DISSOLVED_REACTIVE_PHOSPHORUS']]

print("\n🎯 Predictor Variables (Features):")
for i, col in enumerate(predictor_cols, 1):
    print(f"  {i}. {col}")

print("\n🎯 Target Variables (Response):")
for i, col in enumerate(target_cols, 1):
    print(f"  {i}. {col}")

print("\n" + "=" * 80)

# ============================================
# Step 5: Display sample and summary
# ============================================
print("\nSample of selected dataset (first 5 rows):")
print("=" * 80)
display(wq_data.head())

print("\n" + "=" * 80)
print("Summary statistics:")
print("=" * 80)
display(wq_data.describe())

print("\n" + "=" * 80)
print("✅ DATA IS READY FOR TRAIN/TEST SPLIT AND MODEL TRAINING!")
print("=" * 80)

### **Tip 3**

We are developing individual models for each water quality parameter using a common set of features: **SWIR22**, **NDMI**, **MNDWI**, and **PET**. However, participants are encouraged to experiment with different feature combinations to build more robust machine learning models.


## Helper Functions

### Train and Test Split
We will now split the data into 70% training data and 30% test data. Scikit-learn (sklearn) is a robust library for machine learning in Python. The `model_selection` module in scikit-learn provides the `train_test_split` function, which can be used for this purpose.

### Feature Scaling
Before initiating model training, we may need to perform various data preprocessing steps. Here, we demonstrate scaling of the variables **SWIR22**, **NDMI**, **MNDWI**, and **PET** using StandardScaler.

Feature scaling is an essential preprocessing step for numerical features. Many machine learning algorithms—such as gradient descent methods, KNN, and linear or logistic regression—require scaling to achieve optimal performance. Scikit-learn provides several scaling utilities. In this notebook, we use **StandardScaler**, which transforms the data so that each feature has a mean of 0 and a standard deviation of 1.

### Model Training
Now that we have the data in a format suitable for machine learning, we can begin training our models. In this demonstration notebook, we build three separate regression models—one for each target water quality parameter: **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus**. Each model is trained independently to capture the unique relationships between the satellite-derived features and each water quality parameter.

We use the **Random Forest Regressor** from the scikit-learn library for model training. Scikit-learn offers a wide range of regression algorithms, along with powerful parameter tuning and customization options.

For model training, the predictor variables (e.g., SWIR22, NDMI, MNDWI, and PET) are stored in an array `X`, and the response variable (one of the water quality parameters) is stored in an array `Y`. Note that the response variable should not be included in `X`. Also, latitude, longitude, and sample date are excluded from the predictor variables since they serve only as spatial and temporal references.

### Model Evaluation
After training the models for the three water quality parameters, the next step is to evaluate their performance. Each regression model—Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus—is assessed using:

- **R² Score**: Measures how well the model explains the variance in the observed values.  
- **RMSE (Root Mean Square Error)**: Quantifies the average magnitude of prediction errors.

Together, these metrics help determine how effectively each model captures variations in water quality across locations and sampling dates. Scikit-learn provides built-in functions to compute both metrics. Participants may also explore additional evaluation techniques or custom metrics to enhance model assessment.


In [ ]:
# ============================================
# Helper Functions
# ============================================
print("=" * 80)
print("🔧 DEFINING HELPER FUNCTIONS")
print("=" * 80)

print("\nWe will now split the data into 70% training data and 30% test data.")
print("Scikit-learn (sklearn) is a robust library for machine learning in Python.")
print("The scikit-learn library has a model_selection module in which there is a")
print("splitting function train_test_split. You can use the same.")

print("\n" + "=" * 80)

# Function 1: Split data into training and test sets
def split_data(X, y, test_size=0.3, random_state=42):
    """
    Split data into training and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

print("✅ Function 1: split_data() - Splits data into 70% train, 30% test")

# Function 2: Scale features using StandardScaler
def scale_data(X_train, X_test):
    """
    Scale features using StandardScaler.
    Transforms data so each feature has mean of 0 and standard deviation of 1.
    """
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

print("✅ Function 2: scale_data() - Scales features using StandardScaler")

# Function 3: Train a Random Forest model
def train_model(X_train_scaled, y_train):
    """
    Train a Random Forest Regressor model.
    """
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    return model

print("✅ Function 3: train_model() - Trains Random Forest Regressor")

# Function 4: Evaluate model performance
def evaluate_model(model, X_scaled, y_true, dataset_name="Test"):
    """
    Evaluate model and return predictions, R² score, and RMSE.
    """
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    print(f"\n{dataset_name} Evaluation:")
    print(f"  R²: {r2:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    
    return y_pred, r2, rmse

print("✅ Function 4: evaluate_model() - Evaluates model with R² and RMSE")

print("\n" + "=" * 80)
print("✅ All helper functions defined successfully!")
print("=" * 80)

### **Tip 4**

There are many data preprocessing methods available that may help improve model performance. Participants are encouraged to explore various preprocessing techniques as well as different machine learning algorithms to build a more robust model.


In [ ]:
# ============================================
# Train and Test Split
# ============================================
print("=" * 80)
print("📊 TRAIN AND TEST SPLIT")
print("=" * 80)

print("\nWe will now split the data into 70% training data and 30% test data.")
print("Scikit-learn alias 'sklearn' is a robust library for machine learning in Python.")
print("The scikit-learn library has a model_selection module in which there is a")
print("splitting function train_test_split.")

print("\n" + "=" * 80)

# Define predictor variables (X) and target variables (y)
predictor_cols = ['SWIR22', 'NDMI', 'MNDWI', 'PET']
target_cols = ['TOTAL_ALKALINITY', 'ELECTRICAL_CONDUCTANCE', 'DISSOLVED_REACTIVE_PHOSPHORUS']

# Check if columns exist
available_predictors = [col for col in predictor_cols if col in wq_data.columns]
available_targets = [col for col in target_cols if col in wq_data.columns]

print("\n🎯 Predictor Variables (X):")
for col in available_predictors:
    print(f"  • {col}")

print("\n🎯 Target Variables (y):")
for col in available_targets:
    print(f"  • {col}")

# Extract features (X) and targets (y)
X = wq_data[available_predictors]
y_ta = wq_data['TOTAL_ALKALINITY'] if 'TOTAL_ALKALINITY' in wq_data.columns else None
y_ec = wq_data['ELECTRICAL_CONDUCTANCE'] if 'ELECTRICAL_CONDUCTANCE' in wq_data.columns else None
y_drp = wq_data['DISSOLVED_REACTIVE_PHOSPHORUS'] if 'DISSOLVED_REACTIVE_PHOSPHORUS' in wq_data.columns else None

print("\n" + "=" * 80)

# Split data for each target variable
print("\n🔀 Splitting data (70% train, 30% test)...")

# Total Alkalinity (TA)
X_train_ta, X_test_ta, y_train_ta, y_test_ta = split_data(X, y_ta, test_size=0.3, random_state=42)
print(f"\n✅ Total Alkalinity:")
print(f"   Training set: {X_train_ta.shape[0]} samples")
print(f"   Test set: {X_test_ta.shape[0]} samples")

# Electrical Conductance (EC)
X_train_ec, X_test_ec, y_train_ec, y_test_ec = split_data(X, y_ec, test_size=0.3, random_state=42)
print(f"\n✅ Electrical Conductance:")
print(f"   Training set: {X_train_ec.shape[0]} samples")
print(f"   Test set: {X_test_ec.shape[0]} samples")

# Dissolved Reactive Phosphorus (DRP)
X_train_drp, X_test_drp, y_train_drp, y_test_drp = split_data(X, y_drp, test_size=0.3, random_state=42)
print(f"\n✅ Dissolved Reactive Phosphorus:")
print(f"   Training set: {X_train_drp.shape[0]} samples")
print(f"   Test set: {X_test_drp.shape[0]} samples")

print("\n" + "=" * 80)
print("✅ Data split complete!")
print("=" * 80)

In [9]:
# ============================================
# Feature Scaling (Tip 4)
# ============================================
print("=" * 80)
print("💡 TIP 4: FEATURE SCALING")
print("=" * 80)

print("\nBefore initiating model training, we may need to perform various data")
print("preprocessing steps. Here, we demonstrate scaling of the variables SWIR22,")
print("NDMI, MNDWI, and PET using StandardScaler.")

print("\nFeature scaling is an essential preprocessing step for numerical features.")
print("Many machine learning algorithms—such as gradient descent methods, KNN, and")
print("linear or logistic regression—require scaling to achieve optimal performance.")
print("Scikit-learn provides several scaling utilities. In this notebook, we use")
print("StandardScaler, which transforms the data so that each feature has a mean of")
print("0 and a standard deviation of 1.")

print("\n" + "=" * 80)

# Scale features for Total Alkalinity model
print("\n🔧 Scaling features for Total Alkalinity model...")
X_train_ta_scaled, X_test_ta_scaled, scaler_ta = scale_data(X_train_ta, X_test_ta)
print(f"✅ TA features scaled")

# Scale features for Electrical Conductance model
print("\n🔧 Scaling features for Electrical Conductance model...")
X_train_ec_scaled, X_test_ec_scaled, scaler_ec = scale_data(X_train_ec, X_test_ec)
print(f"✅ EC features scaled")

# Scale features for Dissolved Reactive Phosphorus model
print("\n🔧 Scaling features for Dissolved Reactive Phosphorus model...")
X_train_drp_scaled, X_test_drp_scaled, scaler_drp = scale_data(X_train_drp, X_test_drp)
print(f"✅ DRP features scaled")

print("\n" + "=" * 80)
print("✅ All features scaled successfully!")
print("=" * 80)

print("\n📊 Feature scaling complete. Each feature now has:")
print("  • Mean = 0")
print("  • Standard Deviation = 1")

In [ ]:
# ============================================
# Model Training (Tip 3)
# ============================================
print("=" * 80)
print("🤖 MODEL TRAINING")
print("=" * 80)

print("\n💡 TIP 3: INDIVIDUAL MODELS FOR EACH PARAMETER")
print("=" * 80)

print("\nNow that we have the data in a format suitable for machine learning, we can")
print("begin training our models. In this demonstration notebook, we build three")
print("separate regression models—one for each target water quality parameter:")
print("  • Total Alkalinity")
print("  • Electrical Conductance")
print("  • Dissolved Reactive Phosphorus")

print("\nEach model is trained independently to capture the unique relationships between")
print("the satellite-derived features and each water quality parameter.")

print("\nWe use the Random Forest Regressor from the scikit-learn library for model")
print("training. Scikit-learn offers a wide range of regression algorithms, along with")
print("powerful parameter tuning and customization options.")

print("\nFor model training, the predictor variables (e.g., SWIR22, NDMI, MNDWI, and PET)")
print("are stored in an array X, and the response variable (one of the water quality")
print("parameters) is stored in an array y. Note that the response variable should not")
print("be included in X. Also, latitude, longitude, and sample date are excluded from")
print("the predictor variables since they serve only as spatial and temporal references.")

print("\n" + "=" * 80)

# Train Model 1: Total Alkalinity
print("\n🎯 Training Model 1: Total Alkalinity (TA)")
print("-" * 80)
model_ta = train_model(X_train_ta_scaled, y_train_ta)
print(f"✅ Total Alkalinity model trained with {model_ta.n_estimators} trees")

# Train Model 2: Electrical Conductance
print("\n🎯 Training Model 2: Electrical Conductance (EC)")
print("-" * 80)
model_ec = train_model(X_train_ec_scaled, y_train_ec)
print(f"✅ Electrical Conductance model trained with {model_ec.n_estimators} trees")

# Train Model 3: Dissolved Reactive Phosphorus
print("\n🎯 Training Model 3: Dissolved Reactive Phosphorus (DRP)")
print("-" * 80)
model_drp = train_model(X_train_drp_scaled, y_train_drp)
print(f"✅ Dissolved Reactive Phosphorus model trained with {model_drp.n_estimators} trees")

print("\n" + "=" * 80)
print("✅ ALL THREE MODELS TRAINED SUCCESSFULLY!")
print("=" * 80)

In [ ]:
# ============================================
# Model Evaluation
# ============================================
print("=" * 80)
print("📈 MODEL EVALUATION")
print("=" * 80)

print("\nAfter training the models for the three water quality parameters, the next step")
print("is to evaluate their performance. Each regression model—Total Alkalinity,")
print("Electrical Conductance, and Dissolved Reactive Phosphorus—is assessed using:")

print("\n  • R² Score: Measures how well the model explains the variance in the observed values.")
print("  • RMSE (Root Mean Square Error): Quantifies the average magnitude of prediction errors.")

print("\nTogether, these metrics help determine how effectively each model captures")
print("variations in water quality across locations and sampling dates. Scikit-learn")
print("provides built-in functions to compute both metrics. Participants may also")
print("explore additional evaluation techniques or custom metrics to enhance model")
print("assessment.")

print("\n" + "=" * 80)

# Evaluate Model 1: Total Alkalinity
print("\n🎯 MODEL 1: TOTAL ALKALINITY (TA)")
print("=" * 80)
y_pred_ta, r2_ta, rmse_ta = evaluate_model(model_ta, X_test_ta_scaled, y_test_ta, "Total Alkalinity Test")

# Evaluate Model 2: Electrical Conductance
print("\n🎯 MODEL 2: ELECTRICAL CONDUCTANCE (EC)")
print("=" * 80)
y_pred_ec, r2_ec, rmse_ec = evaluate_model(model_ec, X_test_ec_scaled, y_test_ec, "Electrical Conductance Test")

# Evaluate Model 3: Dissolved Reactive Phosphorus
print("\n🎯 MODEL 3: DISSOLVED REACTIVE PHOSPHORUS (DRP)")
print("=" * 80)
y_pred_drp, r2_drp, rmse_drp = evaluate_model(model_drp, X_test_drp_scaled, y_test_drp, "Dissolved Reactive Phosphorus Test")

print("\n" + "=" * 80)
print("📊 SUMMARY OF MODEL PERFORMANCE")
print("=" * 80)

summary_df = pd.DataFrame({
    'Model': ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus'],
    'R² Score': [r2_ta, r2_ec, r2_drp],
    'RMSE': [rmse_ta, rmse_ec, rmse_drp]
})

display(summary_df)

print("\n" + "=" * 80)
print("✅ MODEL EVALUATION COMPLETE!")
print("=" * 80)

## Model Workflow (Pipeline)

The complete model development process follows a structured pipeline to ensure consistency, reproducibility, and clarity. Each stage in the workflow is modularized into independent functions that can be reused for different water quality parameters. This modular approach streamlines the process and makes the workflow easily adaptable to new datasets or parameters in the future.

The pipeline automates the sequence of steps — from data preparation to evaluation — for each target parameter. The same set of predictor variables is used, while the response variable changes for each of the three targets: *Total Alkalinity (TA)*, *Electrical Conductance (EC)*, and *Dissolved Reactive Phosphorus (DRP)*. By maintaining a consistent framework, comparisons across models remain fair and interpretable.


In [10]:
# ============================================
# Model Workflow (Pipeline)
# ============================================
print("=" * 80)
print("🔄 MODEL WORKFLOW PIPELINE")
print("=" * 80)

print("\nThe complete model development process follows a structured pipeline to ensure")
print("consistency, reproducibility, and clarity. Each stage in the workflow is")
print("modularized into independent functions that can be reused for different water")
print("quality parameters. This modular approach streamlines the process and makes the")
print("workflow easily adaptable to new datasets or parameters in the future.")

print("\nThe pipeline automates the sequence of steps — from data preparation to")
print("evaluation — for each target parameter. The same set of predictor variables is")
print("used, while the response variable changes for each of the three targets:")
print("  • Total Alkalinity (TA)")
print("  • Electrical Conductance (EC)")
print("  • Dissolved Reactive Phosphorus (DRP)")

print("\nBy maintaining a consistent framework, comparisons across models remain fair")
print("and interpretable.")

print("\n" + "=" * 80)

def run_pipeline(X, y, param_name="Parameter"):
    """
    Complete machine learning pipeline for a single water quality parameter.
    
    Steps:
    1. Split data into train/test (70/30)
    2. Scale features using StandardScaler
    3. Train Random Forest model
    4. Evaluate on training set (in-sample)
    5. Evaluate on test set (out-sample)
    6. Return results as DataFrame
    """
    print(f"\n{'=' * 60}")
    print(f"Training Model for {param_name}")
    print(f"{'=' * 60}")
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # Scale
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)
    
    # Train
    model = train_model(X_train_scaled, y_train)
    
    # Evaluate (in-sample)
    y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")
    
    # Evaluate (out-sample)
    y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")
    
    # Return summary
    results = {
        "Parameter": param_name,
        "R2_Train": r2_train,
        "RMSE_Train": rmse_train,
        "R2_Test": r2_test,
        "RMSE_Test": rmse_test
    }
    
    return model, scaler, pd.DataFrame([results])

print("\n✅ Pipeline function defined successfully!")
print("=" * 80)

In [ ]:
# ============================================
# Run Pipeline for All Water Quality Parameters
# ============================================
print("=" * 80)
print("🚀 RUNNING PIPELINE FOR ALL PARAMETERS")
print("=" * 80)

# Define features (X) and targets (y)
X = wq_data[['SWIR22', 'NDMI', 'MNDWI', 'PET']]
y_ta = wq_data['TOTAL_ALKALINITY']
y_ec = wq_data['ELECTRICAL_CONDUCTANCE']
y_drp = wq_data['DISSOLVED_REACTIVE_PHOSPHORUS']

# Run pipeline for Total Alkalinity
model_ta, scaler_ta, results_ta = run_pipeline(X, y_ta, param_name="Total Alkalinity")

# Run pipeline for Electrical Conductance
model_ec, scaler_ec, results_ec = run_pipeline(X, y_ec, param_name="Electrical Conductance")

# Run pipeline for Dissolved Reactive Phosphorus
model_drp, scaler_drp, results_drp = run_pipeline(X, y_drp, param_name="Dissolved Reactive Phosphorus")

# Combine all results
print("\n" + "=" * 80)
print("📊 FINAL MODEL COMPARISON")
print("=" * 80)

all_results = pd.concat([results_ta, results_ec, results_drp], ignore_index=True)
display(all_results)

print("\n" + "=" * 80)
print("✅ PIPELINE COMPLETE - ALL MODELS TRAINED AND EVALUATED!")
print("=" * 80)

# Summary insights
print("\n📈 Model Performance Insights:")
for idx, row in all_results.iterrows():
    print(f"\n{row['Parameter']}:")
    print(f"  Training R²: {row['R2_Train']:.3f}")
    print(f"  Test R²: {row['R2_Test']:.3f}")
    
    # Check for overfitting
    if row['R2_Train'] - row['R2_Test'] > 0.1:
        print(f"  ⚠️  Possible overfitting detected (Train-Test gap: {row['R2_Train'] - row['R2_Test']:.3f})")
    else:
        print(f"  ✅ Model generalizes well")

### Model Training and Evaluation for Each Parameter

In this step, we apply the complete modeling pipeline to each of the three selected water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. The input feature set (`X`) remains the same across all three models, while the target variable (`y`) changes for each parameter. 

For every parameter, the `run_pipeline()` function is executed, which handles data preprocessing, model training, and both in-sample and out-of-sample evaluation. This ensures a consistent workflow and allows for a fair comparison of model performance across different water quality indicators.


In [11]:
# ============================================
# Run Pipeline for All Three Water Quality Parameters
# ============================================
print("=" * 80)
print("🚀 RUNNING PIPELINE FOR ALL WATER QUALITY PARAMETERS")
print("=" * 80)

print("\nIn this step, we apply the complete modeling pipeline to each of the three")
print("selected water quality parameters — Total Alkalinity, Electrical Conductance,")
print("and Dissolved Reactive Phosphorus. The input feature set (X) remains the same")
print("across all three models, while the target variable (y) changes for each parameter.")

print("\nFor every parameter, the run_pipeline() function is executed, which handles")
print("data preprocessing, model training, and both in-sample and out-of-sample")
print("evaluation. This ensures a consistent workflow and allows for a fair comparison")
print("of model performance across different water quality indicators.")

print("\n" + "=" * 80)

# First, check what columns we actually have
print("\n🔍 Available columns in wq_data:")
print(wq_data.columns.tolist())

print("\n" + "=" * 80)

# Define feature set (X) - use correct column names (UPPERCASE with underscores)
X = wq_data[['SWIR22', 'NDMI', 'MNDWI', 'PET']]

# Define target variables (y) for each parameter - use correct column names
y_TA = wq_data['TOTAL_ALKALINITY']
y_EC = wq_data['ELECTRICAL_CONDUCTANCE']
y_DRP = wq_data['DISSOLVED_REACTIVE_PHOSPHORUS']

print("\n🎯 Feature set (X) shape:", X.shape)
print("   Features:", X.columns.tolist())

print("\n🎯 Target variables:")
print(f"   y_TA (Total Alkalinity): {y_TA.shape[0]} samples")
print(f"   y_EC (Electrical Conductance): {y_EC.shape[0]} samples")
print(f"   y_DRP (Dissolved Reactive Phosphorus): {y_DRP.shape[0]} samples")

print("\n" + "=" * 80)

# Run pipeline for Total Alkalinity
print("\n📊 PARAMETER 1: TOTAL ALKALINITY")
model_TA, scaler_TA, results_TA = run_pipeline(X, y_TA, "Total Alkalinity")

# Run pipeline for Electrical Conductance
print("\n📊 PARAMETER 2: ELECTRICAL CONDUCTANCE")
model_EC, scaler_EC, results_EC = run_pipeline(X, y_EC, "Electrical Conductance")

# Run pipeline for Dissolved Reactive Phosphorus
print("\n📊 PARAMETER 3: DISSOLVED REACTIVE PHOSPHORUS")
model_DRP, scaler_DRP, results_DRP = run_pipeline(X, y_DRP, "Dissolved Reactive Phosphorus")

print("\n" + "=" * 80)
print("✅ ALL PIPELINES EXECUTED SUCCESSFULLY!")
print("=" * 80)

# Combine all results into one DataFrame
all_results = pd.concat([results_TA, results_EC, results_DRP], ignore_index=True)

print("\n" + "=" * 80)
print("📊 FINAL MODEL PERFORMANCE SUMMARY")
print("=" * 80)
display(all_results)

print("\n" + "=" * 80)
print("📈 MODEL PERFORMANCE ANALYSIS")
print("=" * 80)

for idx, row in all_results.iterrows():
    print(f"\n{row['Parameter']}:")
    print(f"  ✓ Training R²: {row['R2_Train']:.3f} | RMSE: {row['RMSE_Train']:.3f}")
    print(f"  ✓ Test R²: {row['R2_Test']:.3f} | RMSE: {row['RMSE_Test']:.3f}")
    
    # Check for overfitting
    r2_gap = row['R2_Train'] - row['R2_Test']
    if r2_gap > 0.1:
        print(f"  ⚠️  Warning: Possible overfitting (R² gap: {r2_gap:.3f})")
    else:
        print(f"  ✅ Model generalizes well (R² gap: {r2_gap:.3f})")

print("\n" + "=" * 80)
print("✨ MODEL TRAINING AND EVALUATION COMPLETE!")
print("=" * 80)

# Store models and scalers for later use
print("\n💾 Trained models and scalers saved:")
print("   • model_TA, scaler_TA - Total Alkalinity")
print("   • model_EC, scaler_EC - Electrical Conductance")
print("   • model_DRP, scaler_DRP - Dissolved Reactive Phosphorus")

### Model Performance Summary

After training and evaluating the models for each water quality parameter, the individual performance metrics are combined into a single summary table. This table consolidates the R² and RMSE values for both in-sample and out-of-sample evaluations, enabling an easy comparison of model performance across Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. 

Such a summary provides a quick overview of how well each model captures the variability in each parameter and highlights any differences in predictive accuracy.


In [12]:
# ============================================
# IMPROVED MODEL TRAINING WITH MORE FEATURES
# ============================================
print("=" * 80)
print("🚀 IMPROVED PIPELINE WITH ANTI-OVERFITTING MEASURES")
print("=" * 80)

print("\n💡 Improvements Applied:")
print("  1. Using MORE features (not just 4)")
print("  2. Better Random Forest hyperparameters to reduce overfitting")
print("  3. Regularization through max_depth and min_samples constraints")

print("\n" + "=" * 80)

# Step 1: Collect ALL available features
available_features = []

# Must-have features
must_have = ['SWIR22', 'NDMI', 'MNDWI', 'PET']
for feat in must_have:
    if feat in wq_data.columns:
        available_features.append(feat)

# Optional additional features (spectral bands)
optional = ['NIR', 'GREEN', 'RED', 'BLUE', 'SWIR16', 'SWIR1', 'SWIR2']
for feat in optional:
    if feat in wq_data.columns and feat not in available_features:
        available_features.append(feat)

print(f"\n🎯 Using {len(available_features)} features:")
for i, feat in enumerate(available_features, 1):
    print(f"  {i}. {feat}")

# Create feature matrix
X = wq_data[available_features]

# Define targets
y_TA = wq_data['TOTAL_ALKALINITY']
y_EC = wq_data['ELECTRICAL_CONDUCTANCE']
y_DRP = wq_data['DISSOLVED_REACTIVE_PHOSPHORUS']

print("\n" + "=" * 80)

# Define improved training function
def train_model_improved(X_train_scaled, y_train):
    """Random Forest with anti-overfitting settings"""
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)
    return model

# Define improved pipeline
def run_pipeline_improved(X, y, param_name="Parameter"):
    """Complete ML pipeline with improved model"""
    print(f"\n{'=' * 60}")
    print(f"Training Improved Model for {param_name}")
    print(f"{'=' * 60}")
    
    # Split
    X_train, X_test, y_train, y_test = split_data(X, y)
    print(f"  Train: {X_train.shape[0]} samples | Test: {X_test.shape[0]} samples")
    
    # Scale
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)
    
    # Train with improved settings
    model = train_model_improved(X_train_scaled, y_train)
    
    # Evaluate
    y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")
    y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")
    
    # Return results
    results = {
        "Parameter": param_name,
        "R2_Train": r2_train,
        "RMSE_Train": rmse_train,
        "R2_Test": r2_test,
        "RMSE_Test": rmse_test
    }
    
    return model, scaler, pd.DataFrame([results])

# Run improved pipeline for all parameters
print("\n📊 TRAINING IMPROVED MODELS...")
print("=" * 80)

model_TA_v2, scaler_TA_v2, results_TA_v2 = run_pipeline_improved(X, y_TA, "Total Alkalinity")
model_EC_v2, scaler_EC_v2, results_EC_v2 = run_pipeline_improved(X, y_EC, "Electrical Conductance")
model_DRP_v2, scaler_DRP_v2, results_DRP_v2 = run_pipeline_improved(X, y_DRP, "Dissolved Reactive Phosphorus")

# Combine results
results_summary_improved = pd.concat([results_TA_v2, results_EC_v2, results_DRP_v2], ignore_index=True)

print("\n" + "=" * 80)
print("📊 IMPROVED MODEL PERFORMANCE")
print("=" * 80)
display(results_summary_improved)

print("\n" + "=" * 80)
print("📈 COMPARISON: OLD vs NEW")
print("=" * 80)

comparison = pd.DataFrame({
    'Parameter': results_summary['Parameter'],
    'Old_R2_Test': results_summary['R2_Test'],
    'New_R2_Test': results_summary_improved['R2_Test'],
    'Improvement': results_summary_improved['R2_Test'] - results_summary['R2_Test'],
    'Old_Gap': results_summary['R2_Train'] - results_summary['R2_Test'],
    'New_Gap': results_summary_improved['R2_Train'] - results_summary_improved['R2_Test']
})

display(comparison)

print("\n" + "=" * 80)
print("✨ IMPROVEMENTS APPLIED SUCCESSFULLY!")
print("=" * 80)

for idx, row in comparison.iterrows():
    print(f"\n{row['Parameter']}:")
    print(f"  Old R² Test: {row['Old_R2_Test']:.4f} | New R² Test: {row['New_R2_Test']:.4f}")
    print(f"  Improvement: {row['Improvement']:+.4f}")
    print(f"  Overfitting Gap: {row['Old_Gap']:.4f} → {row['New_Gap']:.4f}")
    
    if row['New_Gap'] < row['Old_Gap']:
        print(f"  ✅ Overfitting REDUCED by {(row['Old_Gap'] - row['New_Gap']):.4f}")

## Submission

Once you are satisfied with your model’s performance, you can proceed to make predictions for unseen data. To do this, use your trained model to estimate the concentrations of the target water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus — for a set of test locations provided in the **Submission_template.csv** file. 

The predicted results can then be uploaded to the challenge platform for evaluation.


In [ ]:
test_file = pd.read_csv("submission_template.csv")
display(test_file.head(5))

In [ ]:
landsat_val_features = pd.read_csv("landsat_features_validation.csv")
display(landsat_val_features.head(5))

In [ ]:
Terraclimate_val_df = pd.read_csv("terraclimate_features_validation.csv")
display(Terraclimate_val_df.head(5))

Similarly, participants can use the **Landsat** and **TerraClimate** data extraction demonstration notebooks to produce feature CSVs for their **validation** data. For convenience, we have already computed and saved example validation outputs as `landsat_features_val_V3.csv` and `Terraclimate_val_df_v3.csv`. 

Participants should save their own extracted files in the same format and column schema; doing so will allow this benchmark notebook to load the validation features directly and run smoothly.


In [16]:
#Consolidate all the extracted bands and features in a single dataframe
val_data = pd.DataFrame({
    'Longitude': landsat_val_features['Longitude'].values,
    'Latitude': landsat_val_features['Latitude'].values,
    'Sample Date': landsat_val_features['Sample Date'].values,
    'nir': landsat_val_features['nir'].values,
    'green': landsat_val_features['green'].values,
    'swir16': landsat_val_features['swir16'].values,
    'swir22': landsat_val_features['swir22'].values,
    'NDMI': landsat_val_features['NDMI'].values,
    'MNDWI': landsat_val_features['MNDWI'].values,
    'pet': Terraclimate_val_df['pet'].values,
})

In [17]:
# Impute the missing values
val_data = val_data.fillna(val_data.median(numeric_only=True))

In [ ]:
# Extracting specific columns (swir22, NDMI, MNDWI, pet) from the validation dataset
submission_val_data=val_data.loc[:,['swir22','NDMI','MNDWI','pet']]
display(submission_val_data.head())

In [19]:
submission_val_data.shape

In [20]:
# --- Predicting for Total Alkalinity ---
X_sub_scaled_TA = scaler_TA.transform(submission_val_data)
pred_TA_submission = model_TA.predict(X_sub_scaled_TA)

# --- Predicting for Electrical Conductance ---
X_sub_scaled_EC = scaler_EC.transform(submission_val_data)
pred_EC_submission = model_EC.predict(X_sub_scaled_EC)

# --- Predicting for Dissolved Reactive Phosphorus ---
X_sub_scaled_DRP = scaler_DRP.transform(submission_val_data)
pred_DRP_submission = model_DRP.predict(X_sub_scaled_DRP)

In [21]:
submission_df = pd.DataFrame({
    'Longitude': test_file['Longitude'].values,
    'Latitude': test_file['Latitude'].values,
    'Sample Date': test_file['Sample Date'].values,
    'Total Alkalinity': pred_TA_submission,
    'Electrical Conductance': pred_EC_submission,
    'Dissolved Reactive Phosphorus': pred_DRP_submission
})

In [22]:
#Displaying the sample submission dataframe
display(submission_df.head())

In [23]:
#Dumping the predictions into a csv file.
submission_df.to_csv("/tmp/submission.csv",index = False)

In [ ]:
session.sql(f"""
    PUT file:///tmp/submission.csv
    snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

### Upload submission file on platform

Upload the `submission.csv` file on the challenge platform to generate your score on the leaderboard.


## Conclusion

Now that you have learned a basic approach to model training, it’s time to explore your own techniques and ideas! Feel free to modify any of the functions presented in this notebook to experiment with alternative preprocessing steps, feature engineering strategies, or machine learning algorithms. 

We look forward to seeing your enhanced model and the insights you uncover. Best of luck with the challenge!
